## 简介
进行因子的测试，使用国金的一个回测方法：
1. 2800只股票按照行业进行分组，在每个行业内分x组，之后再分投资组，每组内行业配置权重使用沪深300；
2. 卖空沪深300期指进行对冲；

In [ ]:
# 载入zipline，用于之后的%%zipline
import zipline
%load_ext zipline
%matplotlib inline

In [ ]:
%%zipline -b my_bundle_china --start 2016-3-4 --end 2016-3-25 --data-frequency daily --capital-base 10000000

from zipline.api import order, symbol, set_long_only, get_open_orders, get_datetime, cancel_order, schedule_function, date_rules, order_target_percent
import tushare as ts
from jingong_factor_access.get_factor import FactorAccessTCsv, FactorAccessTMysql
import pandas as pd

def generate_group(group_num, hs300_weight_pd, stock_industry_pd, stock_factor_pd):
    """
    生成股票投资分组，划分为group_num个组，每组内行业的投资权重与沪深300相同
    :param group_num: 分组数
    :param hs300_weight_pd: 沪深300股票权重, hs300_weight = ts.get_hs300s()
    :param stock_industry_pd: 股票行业列表, FactorAccessTCsv.get_factor_values_all('stock_industry_shenwan_no1')
    :param stock_factor_pd: 股票因子列表, fa.get_factor_values_all('fea00015', '2016-06-06', '2016-06-06')
    """
    # 计算沪深300行业权重
    hs300_indust_weight = dict()
    for i in hs300_weight_pd.index:
        stock_code = hs300_weight_pd.loc[i, 'code']
        stock_weight = hs300_weight_pd.loc[i, 'weight']
        stock_indust = stock_industry_pd[stock_industry_pd.stock_code==stock_code]['industry_name'].values[0]
        if stock_indust not in hs300_indust_weight.keys():
            hs300_indust_weight[stock_indust] = 0.0
        hs300_indust_weight[stock_indust] += stock_weight

    # 根据因子和沪深300行业权重分组
    stock_indust_factor_pd = pd.merge(
        left=stock_industry_pd, right=stock_factor_pd, how='left', left_on='stock_code', right_on='stock_code'
    ).dropna(how='any')
    # 保持分组结果{ 分组1:[(股票1, 权重1), (股票2, 权重2), ...], 分组2:...}
    invest_group_dict = dict()
    for i in range(1, group_num+1):
        invest_group_dict[i] = dict()
        for indust_name in hs300_indust_weight.keys():
            one_indust = stock_indust_factor_pd[stock_indust_factor_pd.industry_name == indust_name]
            one_indust['factor_value'] = one_indust['factor_value'].astype(float)
            one_indust = one_indust.sort_values(by='factor_value', ascending=False)
            # 每组当前行业的股票数
            group_indust_stock_num = int(len(one_indust) / group_num)
            # 每组当前行业股票的投资权重
            stock_weight = hs300_indust_weight[indust_name] / group_indust_stock_num
            for j in range((i-1)*group_indust_stock_num, i*group_indust_stock_num):
                invest_group_dict[i][one_indust.iloc[j]['stock_code']] = stock_weight

    return invest_group_dict


def cancel_all_open_orders():
    """
    根据zipline的get_open_orders返回值取消其所有订单
    """
    from zipline.api import get_open_orders, cancel_order
    for myorder in get_open_orders().values():
        for myevent in myorder:
            cancel_order(myevent.id)

def portfolio_reallocation(context=None, data=None):
    """
    调仓
    """
    # 取消所有订单
    cancel_all_open_orders()
    # 计算新持仓
    # 股票因子
    fa = FactorAccessTMysql()
    stock_factor = fa.get_factor_values_all('fea00015', get_datetime().to_datetime(), get_datetime().to_datetime())
    group_invest = generate_group(5, context.hs300_weight, context.stock_ind, stock_factor)
    
    # 清仓不在投资组合里的股票
    for stk in context.portfolio.positions:
        if stk.symbol not in group_invest[1].keys():
            order_target_percent(stk, 0.0)
    
    # 投资新投资组合
    for stock_code in group_invest[1].keys():
        order_target_percent(symbol(stock_code), group_invest[1][stock_code]/100.0)
    
    
# 回测驱动代码
def initialize(context):
    # TODO 设置zipline.api.set_cancel_policy，单据取消策略，好像现在只能手动取消单据
    # TODO 设置slipp model
    # TODO 设置trading controls, 包括不可交易股票列表、是否只能买多
    
    # 设置回测参数
    # Set a rule specifying that this algorithm cannot take short positions.
    set_long_only()
    # 设置账户的杠杆率，如果为1，表示不允许借贷
    set_max_leverage(1)
    
    # 设置需要使用的上下文变量
    # 股票所属行业
    context.stock_ind = FactorAccessTCsv.get_factor_values_all('stock_industry_shenwan_no1')
    # 沪深300权重股
    context.hs300_weight = ts.get_hs300s()
    
    # 注册换仓函数
    schedule_function(portfolio_reallocation, date_rules.week_start(days_offset=1))

    
def handle_data(context, data):
    pass
        

def analyze(context=None, perf=None):
    import matplotlib.pyplot as plt

    ax1 = plt.subplot(211)
    perf.portfolio_value.plot(ax=ax1)
    ax1.set_ylabel('portfolio value')
    ax2 = plt.subplot(212, sharex=ax1)
    perf.ending_cash.plot(ax=ax2)
    ax2.set_ylabel('ending cash')